In [15]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Set `PATH` to include the directory containing saved_model_cli
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/home/jupyter/.local/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


In [16]:
#!pip uninstall -y tensorflow tensorflow-text transformers

In [17]:
#!pip install tensorflow==2.15.0 tensorflow_text==2.15.0 transformers==4.51.0

In [18]:
import datetime
import shutil

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_text as text

#from official.nlp import optimization
tf.get_logger().setLevel("ERROR")

In [19]:
!git clone https://github.com/eminedemirbas/Hackathon-YZTA

fatal: destination path 'Hackathon-YZTA' already exists and is not an empty directory.


In [20]:
repo_path = "/content/Hackathon-YZTA"


data_path = os.path.join(repo_path, 'data')


if os.path.exists(data_path):
    print(f"Data klasörü bulundu: {data_path}")
else:
    print("Data klasörü bulunamadı.")

Data klasörü bulundu: /content/Hackathon-YZTA/data


In [21]:
# Veriyi yüklemek için klasör yolunu kullanacağız
train_dir = os.path.join(data_path, 'train')
test_dir = os.path.join(data_path, 'test')

In [22]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=seed,
)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=seed,
)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir, batch_size=batch_size
)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 3443 files belonging to 2 classes.
Using 2755 files for training.
Found 3443 files belonging to 2 classes.
Using 688 files for validation.
Found 1000 files belonging to 2 classes.


In [23]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        # Byte formatındaki veriyi UTF-8'e çevir
        review_text = text_batch.numpy()[i].decode('utf-8')
        print(f"Review:  {review_text}")
        label = label_batch.numpy()[i]
        print(f"Label : {label} ({class_names[label]})")

Review:  Kişi bugün evden işe giderken 35 km’lik mesafeyi özel aracıyla katederek 7.2 kg CO2 salımına neden oldu. Yolculuk boyunca araçta tek başınaydı. Gıda alışverişinde et ürünleri ve ithal meyveler alındı. Akşam yemeği hazır gıdalardan oluştu ve yoğun plastik ambalaj kullanımı vardı. Evdeki çamaşır makinesi düşük dolulukla çalıştırıldı. Isıtıcılar yüksek ayarda uzun süre çalıştı. Işıklar kullanılmayan odalarda da açık bırakıldı. Atıkların ayrıştırılması ihmal edildi. Gün boyunca yapılan bu tercihler toplamda 15 kg CO2 emisyonuna yol açtı. Bu davranışların doğaya etkisi ciddi düzeyde olumsuzdur.


Label : 0 (neg)
Review:  Sabah, kısa mesafeye arabayla gitmen çevreye büyük bir zarar verdi. Yolda yürümek veya bisiklet kullanmak, çevreyi korumak adına çok daha iyi bir seçenek olabilirdi. Öğle yemeğinde dışarıda ambalajlı yiyecekler tükettin, bu da doğaya zarar veren atıkların artmasına yol açtı. Akşam alışveriş için arabanı kullandın, gereksiz yakıt harcadın. Çamaşırları yüksek sıcaklı

In [24]:
import transformers
import keras

print("Transformers sürümü:", transformers.__version__)
print("Keras sürümü:", keras.__version__)


Transformers sürümü: 4.51.3
Keras sürümü: 3.8.0


In [25]:
from transformers import TFAutoModel, AutoTokenizer

model_name = "dbmdz/bert-base-turkish-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = TFAutoModel.from_pretrained(model_name)

Some layers from the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [26]:
text_test = ["Çevreye zarar verdin!"]

inputs = tokenizer(text_test, padding=True, truncation=True, return_tensors="tf")

print(f"Keys       : {list(inputs.keys())}")

print(f"Shape      : {inputs['input_ids'].shape}")
print(f"Word Ids   : {inputs['input_ids'][0, :12]}")

print(f"Attention Mask : {inputs['attention_mask'][0, :12]}")

Keys       : ['input_ids', 'token_type_ids', 'attention_mask']
Shape      : (1, 8)
Word Ids   : [    2  3231 20834  1028  3840 19988     5     3]
Attention Mask : [1 1 1 1 1 1 1 1]


In [27]:
bert_results = bert_model(**inputs)

In [28]:
print(f"Loaded BERT model : {bert_model}")

print(f'Pooled Outputs Shape: {bert_results.pooler_output.shape}')
print(f'Pooled Outputs Values: {bert_results.pooler_output[0, :12]}')

print(f'Sequence Outputs Shape: {bert_results.last_hidden_state.shape}')
print(f'Sequence Outputs Values: {bert_results.last_hidden_state[0, :12, :]}')

Loaded BERT model : <transformers.models.bert.modeling_tf_bert.TFBertModel object at 0x7fa6c4c19950>
Pooled Outputs Shape: (1, 768)
Pooled Outputs Values: [ 0.06398559 -0.3753235   0.34511548  0.22325964 -0.9989462   0.9971525
 -0.2679233   0.20474148  0.06282986 -0.99504286 -0.7965103  -0.09152508]
Sequence Outputs Shape: (1, 8, 768)
Sequence Outputs Values: [[-0.45390198  0.04856572  0.39353645 ...  0.62535435  0.4778933
  -0.9104075 ]
 [ 0.25148723 -0.13441703  0.12725598 ... -0.06474683 -0.371077
  -0.7703699 ]
 [-0.61564076 -0.7248291   0.46139172 ... -0.14295396 -1.0015088
   0.38154262]
 ...
 [-0.81980765  0.01571992  0.05395365 ... -0.21817678 -0.09767374
   0.10209192]
 [-0.5636111   0.16253881  1.0184888  ...  0.25842923  0.20594317
  -0.09941857]
 [-0.6223204   0.35600746  0.56431425 ... -0.36647758 -0.41368166
  -1.0223415 ]]


In [29]:
print(f"Keys       : {list(bert_results.keys())}")

Keys       : ['last_hidden_state', 'pooler_output']


In [30]:
def build_classifier_model(dropout_rate=0.1):

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")

    def preprocess(text):
        return tokenizer(text.numpy(), padding=True, truncation=True, return_tensors="tf")

    preprocessing_layer = tf.keras.layers.Lambda(lambda x: tf.py_function(preprocess, [x], tf.float32))(text_input)

    encoder = bert_model(preprocessing_layer)

    net = encoder.pooler_output
    net = tf.keras.layers.Dropout(dropout_rate)(net)

    net = tf.keras.layers.Dense(1, activation="sigmoid", name="classifier")(net)

    return tf.keras.Model(text_input, net)

dropout_rate = 0.15
classifier_model = build_classifier_model(dropout_rate)

text_test = ["Bu gerçekten harika bir film!"]
bert_raw_result = classifier_model(tf.constant(text_test))

print(bert_raw_result)



NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None,), dtype=string, sparse=False, name=text>',)
  • kwargs={'mask': 'None'}

In [ ]:
import sys
print(sys.version)  # Detaylı sürüm bilgisi